In [20]:
import numpy as np
from itertools import product

def transition_probability(nucleotide_from, nucleotide_to, theta):
    if nucleotide_from == nucleotide_to:
        prob = 1/4 + 3/4 * np.exp(-4/3 * theta)
    else:
        prob = 1/4 * (1 - np.exp(-4/3 * theta))
    return prob

def sequence_probability(sequence_from, sequence_to, theta):
    prob_product = 1
    for nucleotide_from, nucleotide_to in zip(sequence_from, sequence_to):
        prob_product *= transition_probability(nucleotide_from, nucleotide_to, theta)
    return prob_product

#Unrooted tree with 3 tips and sequence length i
def calculate_p_D_given_tau_theta(theta, data, i):
    nucleotides = ['T', 'C', 'A', 'G']
    total_prob = 0
    for hidden_state in product(nucleotides, repeat=i):
        hidden_state_list = list(hidden_state)
        p1 = sequence_probability(hidden_state_list, data[0], theta[0])
        p2 = sequence_probability(hidden_state_list, data[1], theta[1])
        p3 = sequence_probability(hidden_state_list, data[2], theta[2])
        total_prob += p1 * p2 * p3
    return total_prob

In [21]:
data = [['T', 'C', 'A'],
        ['T', 'C', 'A'],
        ['T', 'C', 'C']]
theta = [0.001, 0.001, 0.001]

calculate_p_D_given_tau_theta(theta, data, 3)

0.0003305676547972337

In [19]:
calculate_p_D_given_tau_theta(theta, data, 3)

0.0003305676547972337

In [26]:
def calculate_p_D_given_tau_theta_4(theta, data):
    nucleotides = ['T', 'C', 'A', 'G']
    total_prob = 0

    for h11 in nucleotides:
        for h12 in nucleotides:
            for h13 in nucleotides:
                h1 = [h11, h12, h13]
                for h21 in nucleotides:
                    for h22 in nucleotides:
                        for h23 in nucleotides:
                            h2 = [h21, h22, h23]
                            p_hidden = sequence_probability(h1, h2, theta[0])
                            p1 = sequence_probability(h1, data[0], theta[1])
                            p2 = sequence_probability(h1, data[1], theta[2])
                            p3 = sequence_probability(h2, data[2], theta[3])
                            p4 = sequence_probability(h2, data[3], theta[4])
                
                total_prob += p1 * p2 * p3 * p4 * p_hidden
    
    return total_prob

In [28]:
data = [['T', 'C', 'A'],
        ['T', 'C', 'A'],
        ['T', 'C', 'C'],
        ['T', 'C', 'G']]

theta = [0.001, 0.001, 0.001, 0.001, 0.002]
calculate_p_D_given_tau_theta_4(theta, data)

6.013899026898284e-28